In [51]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from get_lat_lon import get_lat_lon_from_address

import tqdm
import requests
from bs4 import BeautifulSoup
import time
import geocoder

In [52]:
# Read recipe inputs
raw_ramen = dataiku.Dataset("ramen_by_store_name")
df = raw_ramen.get_dataframe(limit=100)

In [53]:
df.head()

,store_name,score,review_cnt,store_id,address,address_cleaned,ward,review
0,きたかた食堂,3.66,282,79,東京都港区新橋5-9-8永島ビル 1F,東京都港区5-9-8,東京都内,平朝に伺いました。店内には名のお客さん朝ラー需要ありますね。普段見かけない女性の店員さんと名...
1,つじ田 味噌の章,3.71,418,148,東京都千代田区神田小川町1-1イルヴィアーレ神田小川町ビル 1F,東京都千代田区1-1,東京都内,東京メトロ丸の内線淡路町駅A出口より徒歩約分外堀通り沿いビルFにあるお店つじ田味噌の章。上旬...
2,ねじ式,3.58,151,142,東京都渋谷区幡ヶ谷2-47-12田口ビル1F,東京都渋谷区2-47-12,東京都内,幡ヶ谷号商店街にあるねじ式さんです。いままで味噌ラーメンラーメンしか食べたことが無かったので...
3,ぽっぽっ屋 水道橋店,3.57,186,165,東京都千代田区神田三崎町2-21-11ゑびすビル １Ｆ,東京都千代田区2-21-11,東京都内,ぽっぽ屋水道橋店本当に久しぶりにこちらの麺を頂きました。と言っても水道橋店は初。というかこの...
4,中華そば いしかわや,3.70,186,44,東京都調布市国領町4-8-1,東京都調布市4-8-1,東京都内,旨い醤油ラーメンが食べれる店があるという事でやって来ました。国領駅どういうことで始めておりま...


In [54]:
addresses = df['address_cleaned'].tolist()

In [55]:
def geocoding_lat_lon(address):
    try:
        g = geocoder.mapquest(address, key='eOUDWog4FKpjWQmPZWRCzhiKr3GW0mEr')
        latitude = g.json['raw']['latLng']['lat']
        longitude = g.json['raw']['latLng']['lng']
        return latitude, longitude
    except:
        return np.nan, np.nan

In [56]:
df['latitude'], df['longitude'] = zip(*df['address_cleaned'].map(geocoding_lat_lon))

In [0]:
lat_lon = dataiku.Dataset("lat_lon")
lat_lon.write_with_schema(df)